In [2]:
import numpy as np
import scipy.signal as signal
import soundfile as sf


def a_weighting(fs):
    """Calculate A-weighting filter coefficients.

    Args:
        fs (int): Sampling rate.

    Returns:
        b, a (ndarray): Filter coefficients.
    """
    f1 = 20.598997
    f2 = 107.65265
    f3 = 737.86223
    C1 = 12194.217
    C2 = 20.598997 ** 2 * 737.86223
    C3 = 12194.217 ** 2
    A = [2 * np.pi * f3 * f2, 0, C3 * (2 * np.pi * f1) ** 2 + C1 * (2 * np.pi * f3) ** 2,
         0, 2 * np.pi * f1 * f2 * C1 * C3, C2 * (2 * np.pi * f2) ** 2 + C1 * (2 * np.pi * f1) ** 2]
    d0 = A[0] * A[5] - A[1] * A[4]
    d1 = A[2] * A[5] - A[3] * A[4]
    d2 = A[2] * A[4] - A[3] * A[5]
    b = np.array([A[5] / d0, -A[4] / d0, 0])
    a = np.array([1, -d1 / d0, -d2 / d0])
    return b, a


def calculate_laeq(audio_file_path):
    """Calculate LAeq of the given audio file.

    Args:
        audio_file_path (str): Path to the audio file.

    Returns:
        laeq (float): LAeq value.
    """
    # Load audio file
    audio, fs = sf.read(audio_file_path)

    # Filter with A-weighting
    b, a = a_weighting(fs)
    audio = signal.lfilter(b, a, audio)

    # Calculate LAeq
    block_size = int(fs)
    num_blocks = int(np.ceil(len(audio) / block_size))
    laeqs = []
    for i in range(num_blocks):
        block = audio[i * block_size:(i + 1) * block_size]
        rms = np.sqrt(np.mean(block ** 2))
        laeq = 20 * np.log10(rms / 20e-6)
        laeqs.append(laeq)
    laeq = np.mean(laeqs)

    return laeq


In [3]:
path = r'./input/test/sample.wav'
laeq = calculate_laeq(path)
laeq

37.84080799897417

(array([[-6.10351562e-05, -3.35693359e-04],
        [-9.15527344e-05, -2.44140625e-04],
        [-1.52587891e-04, -1.52587891e-04],
        ...,
        [-3.05175781e-05,  3.96728516e-04],
        [ 6.10351562e-05,  3.35693359e-04],
        [ 1.52587891e-04,  3.35693359e-04]]),
 48000)